In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gensim
from tqdm import tqdm_notebook

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Field, LabelField, BucketIterator, TabularDataset


SEED = 42
np.random.seed(SEED)

In [68]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenizer(text): 
    return [lemmatizer.lemmatize(x) for x in word_tokenize(text)
            if x and x not in stopwords]

In [70]:
df = pd.read_csv('../input/sarcasm/train-balanced-sarcasm.csv')

In [71]:
df2 = pd.concat([df['comment'], df['label']], axis=1)

In [72]:
train, test = train_test_split(df2, test_size=0.2, shuffle=True)

train.to_csv('train.csv', index=None)
test.to_csv('test.csv', index=None)

In [73]:
TEXT = Field(include_lengths=True, batch_first=True, 
             tokenize=tokenizer,
             eos_token='<eos>',
             lower=True,
             stop_words=stopwords
            )
LABEL = LabelField(dtype=tt.int64, use_vocab=True)

dataset_train = TabularDataset('train.csv', format='csv', 
                         fields=[('text', TEXT), ('label', LABEL)],
                         skip_header=True)
test = TabularDataset('test.csv', format='csv', 
                         fields=[('text', TEXT), ('label', LABEL)],
                         skip_header=True)

In [74]:
TEXT.build_vocab(dataset_train, min_freq=5)
len(TEXT.vocab.itos)

36310

In [10]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    '../input/wikinews300d1mvec/wiki-news-300d-1M.vec'
)
weights = tt.FloatTensor(model.vectors)

In [76]:
TEXT.vocab.itos[:10]

['<unk>', '<pad>', '<eos>', '.', ',', "'s", "n't", '?', '!', '...']

In [77]:
LABEL.build_vocab(dataset_train)

In [78]:
train, valid = dataset_train.split(0.8, stratified=True)

In [79]:
np.unique([x.label for x in train.examples], return_counts=True)

(array(['0', '1'], dtype='<U1'), array([323402, 323526]))

In [80]:
np.unique([x.label for x in valid.examples], return_counts=True)

(array(['0', '1'], dtype='<U1'), array([80851, 80881]))

In [81]:
np.unique([x.label for x in test.examples], return_counts=True)

(array(['0', '1'], dtype='<U1'), array([101160, 101006]))

In [82]:
device = tt.device("cuda:0" if tt.cuda.is_available() else "cpu")

In [93]:
class MyModel(nn.Module):
    
    def __init__(self, weights, hidden_size):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(weights, freeze=True)
        
        self.rnn = nn.LSTM(input_size=300,
                           hidden_size=hidden_size,
                           bidirectional=True,
                           batch_first=True
                          )
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2 * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 3)
        )

    def forward(self, batch):
        
        x, x_lengths = batch.text
        x = x.to(device)
        batch.label = batch.label.to(device)
        x = self.embedding(x)

        if x_lengths is not None:
            x_lengths = x_lengths.view(-1).tolist()
            x = nn.utils.rnn.pack_padded_sequence(x, x_lengths, batch_first=True)
        
        _, (hidden, cell) = self.rnn(x)
        
        hidden = hidden.transpose(0,1)
        cell = cell.transpose(0,1)
        hidden = hidden.contiguous().view(hidden.size(0),-1)
        cell = cell.contiguous().view(cell.size(0),-1)
        x = tt.cat([hidden, cell], dim=1).squeeze(1)
        x = self.fc(x)
        return x

In [124]:
tt.cuda.empty_cache()

batch_size = 512

model = MyModel(weights,
                hidden_size=128)
model = model.to(device)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, test),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
criterion = nn.CrossEntropyLoss()

In [125]:
def _train_epoch(model, iterator, optimizer, criterion, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches,
                             desc='epoch %d' % (curr_epoch),
                             leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        pred = model(batch)
        loss = criterion(pred, batch.label)
        loss.backward()
        optimizer.step()
        
        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

def _test_epoch(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            pred = model(batch)
            loss = criterion(pred, batch.label)
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches


def nn_train(model, train_iterator, valid_iterator,
             criterion, optimizer, n_epochs=100,
             scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator,
                                  optimizer, criterion, epoch)
        valid_loss = _test_epoch(model, valid_iterator,
                                 criterion)

        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss,
                  'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'],
                                                                   best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

In [126]:
def nn_test(test_iter):
    model.eval()
    correct = 0

    with tt.no_grad():
        for batch in test_iter:
            output = model(batch)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(batch.label.view_as(pred)).sum().item()
    return 100. * correct / len(test_iter.dataset)

In [127]:
nn_train(model, train_iterator, valid_iterator, 
         criterion, optimizer, scheduler=scheduler, 
         n_epochs=20, early_stopping=2)


validation loss 0.61133



validation loss 0.58904



validation loss 0.58647



validation loss 0.57422



validation loss 0.56851



validation loss 0.56686



validation loss 0.57256



validation loss 0.57205
Early stopping! best epoch: 5 val 0.56686


In [128]:
nn_test(test_iterator)

70.17599398514092